In [1]:
from __future__ import absolute_import, division, print_function
from builtins import *  # @UnusedWildImport

from mcculw import ul
from mcculw.enums import DigitalIODirection
from mcculw.device_info import DaqDeviceInfo

import numpy as np
import pyvisa as visa 
import time as time

In [2]:
try:
    from console_examples_util import config_first_detected_device
except ImportError:
    from .console_examples_util import config_first_detected_device

In [5]:
# search all instrument and assign usb port for power supply #####
rm = visa.ResourceManager() 
inst = rm.list_resources()
power_supply = rm.open_resource(inst[0])
inst
power_supply

<'USBInstrument'('USB0::0x1AB1::0x0E11::DP8D232100126::0::INSTR')>

In [8]:
# DAQ signal and Power Supply read ######
#a=np.linspace(0,3,21)
#power_supply.write(':OUTPut CH1, ON')

def run_example():
    # By default, the example detects and displays all available devices and
    # selects the first device listed. Use the dev_id_list variable to filter
    # detected devices by device ID (see UL documentation for device IDs).
    # If use_device_detection is set to False, the board_num variable needs to
    # match the desired board number configured with Instacal.
    use_device_detection = True
    dev_id_list = []
    board_num = 0

    try:
        if use_device_detection:
            config_first_detected_device(board_num, dev_id_list)

        daq_dev_info = DaqDeviceInfo(board_num)
        if not daq_dev_info.supports_digital_io:
            raise Exception('Error: The DAQ device does not support '
                            'digital I/O')

        print('\nActive DAQ device: ', daq_dev_info.product_name, ' (',
              daq_dev_info.unique_id, ')\n', sep='')

        dio_info = daq_dev_info.get_dio_info()

        # Find the first port that supports input, defaulting to None
        # if one is not found.
        port = next((port for port in dio_info.port_info if port.supports_output),
                    None)
        if not port:
            raise Exception('Error: The DAQ device does not support '
                            'digital output')

        # If the port is configurable, configure it for output.
        if port.is_port_configurable:
            ul.d_config_port(board_num, port.type, DigitalIODirection.OUT)

#         port_value = 0xFF

#         print('Setting', port.type.name, 'to', port_value)

#         # Output the value to the port
#         ul.d_out(board_num, port.type, port_value)

        bit_num = 2 #DIO chanel
        bit_value = 0 #low0, high1
        print('Setting', port.type.name, 'bit', bit_num, 'to', bit_value)

        # Output the value to the bit
        ul.d_bit_out(board_num, port.type, bit_num, bit_value)
        
         

        bit_value = 1 #low0, high1

        # Output the value to the bit
        ul.d_bit_out(board_num, port.type, bit_num, bit_value)
        time.sleep(0.1)
        bit_value = 0 #low0, high1

        # Output the value to the bit
        ul.d_bit_out(board_num, port.type, bit_num, bit_value)

        
    except Exception as e:
        print('\n', e)
    finally:
        if use_device_detection:
            ul.release_daq_device(board_num)

a=np.linspace(0,3,21)
power_supply.write(':OUTPut CH1, ON')
if __name__ == '__main__':
    run_example()
    
for i in range(len(a)):
    power_supply.write(':APPLy CH1,10,' + str(a[i]))
    #power_supply.write(':OUTPut CH1, ON')
    time.sleep(0.001)
# print(a[i])
b=np.linspace(3,0,21)
time.sleep(0.05) 
for i in range(len(b)):
    power_supply.write(':APPLy CH1,10,' + str(b[i]))
    #power_supply.write(':OUTPut CH1, ON')
    time.sleep(0.001)
power_supply.write(':APPLy CH1,10,' + str(0))
time.sleep(5) 
power_supply.write(':APPL Ch1,0,0')
power_supply.write(':OUTP CH1, OFF')


 Error: No DAQ devices found


16

In [9]:
a

array([0.  , 0.15, 0.3 , 0.45, 0.6 , 0.75, 0.9 , 1.05, 1.2 , 1.35, 1.5 ,
       1.65, 1.8 , 1.95, 2.1 , 2.25, 2.4 , 2.55, 2.7 , 2.85, 3.  ])